In [29]:
from concorde.problem import Problem
from concorde.concorde import Concorde
import numpy as np

In [30]:
X = np.load('X_20x20.npy')
Y = np.load('Y_20x20.npy')

In [31]:
a = np.round(X[0],2)
for i in range(a.shape[0]):
    for j in range(i+1, a.shape[0]):
        a[j,i] = a[i,j]

In [32]:
problem = Problem.from_matrix(a)
c = Concorde()


In [35]:
%timeit solution = c.solve(problem, concorde_exe = '/home/vladimir/src/concorde/TSP/concorde')


82.8 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
solution.tour

[0, 6, 9, 4, 3, 12, 8, 19, 17, 15, 10, 13, 1, 16, 18, 11, 14, 5, 7, 2]